In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# For machine learning modeles
from sklearn.cluster import KMeans

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.ensemble import  AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [2]:
df=pd.read_table("/home/riteshkumar/Documents/SA/Assignment_2/Train_Data.txt", sep = ',')
df.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [3]:
df=df.drop(['campaign'],axis=1)
df.head()

,date,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,adgroup 1,ad 2,247,126,1.29,4,925.71


In [4]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse=False)
df.adgroup=ohe.fit_transform(df[['adgroup']])
df.ad=ohe.fit_transform(df[['ad']])
df.head()

,date,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,1.0,1.0,24,6,0.08,0,0.00
1,01-08-2020,0.0,1.0,1,0,0.00,0,0.00
2,01-08-2020,0.0,1.0,13,4,0.04,0,0.00
3,01-08-2020,0.0,1.0,5,4,0.08,0,0.00
4,01-08-2020,1.0,0.0,247,126,1.29,4,925.71


In [5]:
df['date'] = pd.to_datetime(df.date, format='%d-%m-%Y', exact=False)
df.index = df['date']
df['day'] = df.date.dt.day
df['month'] = df.date.dt.month
df['CTR'] = df.clicks*100/df.impressions
df['CPC'] = df.cost/df.clicks
df.head()

,date,adgroup,ad,impressions,clicks,cost,conversions,revenue,day,month,CTR,CPC
date,,,,,,,,,,,,
2020-08-01,2020-08-01,1.0,1.0,24,6,0.08,0,0.00,1,8,25.000000,0.013333
2020-08-01,2020-08-01,0.0,1.0,1,0,0.00,0,0.00,1,8,0.000000,NaN
2020-08-01,2020-08-01,0.0,1.0,13,4,0.04,0,0.00,1,8,30.769231,0.010000
2020-08-01,2020-08-01,0.0,1.0,5,4,0.08,0,0.00,1,8,80.000000,0.020000
2020-08-01,2020-08-01,1.0,0.0,247,126,1.29,4,925.71,1,8,51.012146,0.010238


In [6]:
df=df.drop(['date'],axis=1)
df.head()

,adgroup,ad,impressions,clicks,cost,conversions,revenue,day,month,CTR,CPC
date,,,,,,,,,,,
2020-08-01,1.0,1.0,24,6,0.08,0,0.00,1,8,25.000000,0.013333
2020-08-01,0.0,1.0,1,0,0.00,0,0.00,1,8,0.000000,NaN
2020-08-01,0.0,1.0,13,4,0.04,0,0.00,1,8,30.769231,0.010000
2020-08-01,0.0,1.0,5,4,0.08,0,0.00,1,8,80.000000,0.020000
2020-08-01,1.0,0.0,247,126,1.29,4,925.71,1,8,51.012146,0.010238


In [7]:
df.shape

(4571, 11)

In [8]:
df.isna().sum()

adgroup          0
ad               0
impressions      0
clicks           0
cost             0
conversions      0
revenue          0
day              0
month            0
CTR              0
CPC            444
dtype: int64

In [9]:
X=df.drop(['revenue'],axis=1)
y=df.revenue
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=2)

In [12]:
import xgboost as xg

max_score = 0
max_index = 0
for i in range(1,100):
    xgb_r = xg.XGBRegressor(objective ='reg:squarederror',n_estimators = i, seed = 123)
    xgb_r.fit(X_train,y_train)
    if(xgb_r.score(X_test, y_test) > max_score):
        max_score = xgb_r.score(X_test, y_test)
        max_index = i

xgb_r = xg.XGBRegressor(objective ='reg:squarederror',n_estimators = max_index, seed = 123)
xgb_r.fit(X_train,y_train)
print(xgb_r.score(X_test, y_test))

0.9803710730461584


In [13]:
xgb_r.score(X_train,y_train)

0.9929973724204069

In [14]:
xgb_r.score(X_test,y_test)

0.9803710730461584

In [15]:
new_df=pd.read_table("/home/riteshkumar/Documents/SA/Assignment_2/Test_Data.txt", sep = ',')
new_df.head()

,date,campaign,adgroup,ad,cost,impressions,clicks,conversions
0,01-03-2021,campaign 1,adgroup 1,ad 1,0.58,121,49,1
1,01-03-2021,campaign 1,adgroup 3,ad 1,0.17,22,12,0
2,01-03-2021,campaign 1,adgroup 4,ad 1,0.05,5,3,0
3,01-03-2021,campaign 1,adgroup 2,ad 1,0.01,2,1,0
4,01-03-2021,campaign 1,adgroup 2,ad 2,0.01,3,1,0


In [16]:
new_df=new_df.drop(['campaign'],axis=1)
new_df.head()

,date,adgroup,ad,cost,impressions,clicks,conversions
0,01-03-2021,adgroup 1,ad 1,0.58,121,49,1
1,01-03-2021,adgroup 3,ad 1,0.17,22,12,0
2,01-03-2021,adgroup 4,ad 1,0.05,5,3,0
3,01-03-2021,adgroup 2,ad 1,0.01,2,1,0
4,01-03-2021,adgroup 2,ad 2,0.01,3,1,0


In [17]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(sparse=False)
new_df.adgroup=ohe.fit_transform(new_df[['adgroup']])
new_df.ad=ohe.fit_transform(new_df[['ad']])
new_df.head()

,date,adgroup,ad,cost,impressions,clicks,conversions
0,01-03-2021,1.0,1.0,0.58,121,49,1
1,01-03-2021,0.0,1.0,0.17,22,12,0
2,01-03-2021,0.0,1.0,0.05,5,3,0
3,01-03-2021,0.0,1.0,0.01,2,1,0
4,01-03-2021,0.0,0.0,0.01,3,1,0


In [18]:
new_df['date'] = pd.to_datetime(new_df.date, format='%d-%m-%Y', exact=False)
new_df.index =new_df['date']
new_df['day'] = new_df.date.dt.day
new_df['month'] = new_df.date.dt.month
new_df['CTR'] = new_df.clicks*100/new_df.impressions
new_df['CPC'] = new_df.cost/new_df.clicks
new_df.head()

,date,adgroup,ad,cost,impressions,clicks,conversions,day,month,CTR,CPC
date,,,,,,,,,,,
2021-03-01,2021-03-01,1.0,1.0,0.58,121,49,1,1,3,40.495868,0.011837
2021-03-01,2021-03-01,0.0,1.0,0.17,22,12,0,1,3,54.545455,0.014167
2021-03-01,2021-03-01,0.0,1.0,0.05,5,3,0,1,3,60.000000,0.016667
2021-03-01,2021-03-01,0.0,1.0,0.01,2,1,0,1,3,50.000000,0.010000
2021-03-01,2021-03-01,0.0,0.0,0.01,3,1,0,1,3,33.333333,0.010000


In [19]:
new_df = new_df[X.columns]
new_df.head()

,adgroup,ad,impressions,clicks,cost,conversions,day,month,CTR,CPC
date,,,,,,,,,,
2021-03-01,1.0,1.0,121,49,0.58,1,1,3,40.495868,0.011837
2021-03-01,0.0,1.0,22,12,0.17,0,1,3,54.545455,0.014167
2021-03-01,0.0,1.0,5,3,0.05,0,1,3,60.000000,0.016667
2021-03-01,0.0,1.0,2,1,0.01,0,1,3,50.000000,0.010000
2021-03-01,0.0,0.0,3,1,0.01,0,1,3,33.333333,0.010000


In [20]:
pred = xgb_r.predict(new_df)

In [21]:
print(pred)

[ 1.30590881e+02  5.10152936e-01  5.10152936e-01  7.95906723e-01
  7.95906723e-01  7.95906723e-01  1.87128189e+02  6.64684176e-02
  1.23475685e+02  5.10152936e-01  4.17729797e+02  5.10152936e-01
  7.95906723e-01  3.29292755e+02  5.10152936e-01  1.53568207e+02
  7.95906723e-01  1.23750900e+02  5.10152936e-01  1.16765305e+02
  6.64684176e-02  7.95906723e-01  5.10152936e-01  4.41220093e+02
  6.64684176e-02  7.95906723e-01  4.65594727e+02  1.37266281e+02
  5.10152936e-01  6.64684176e-02  4.38742615e+02  7.95906723e-01
  7.75709656e+02  3.00709717e+02  1.30590881e+02  7.95906723e-01
  1.21839706e+02  7.95906723e-01  6.64684176e-02  5.10152936e-01
  5.10152936e-01  7.95906723e-01  6.64684176e-02  7.95906723e-01
  7.95906723e-01  5.10152936e-01  7.95906723e-01  5.10152936e-01
  5.10152936e-01  7.95906723e-01  6.64684176e-02  5.10152936e-01
 -2.47476792e+00  2.98075470e+02  1.32502075e+02  1.39742630e+02
  5.10152936e-01  7.95906723e-01  7.95906723e-01  7.81440247e+02
  7.95906723e-01  3.45125

In [22]:
res = pd.DataFrame(pred)
res.index = new_df.index 
res.columns = ["revenue"]
res.to_csv("results11.csv", index = False)     